1) Créer le Semantic Layer 

In [1]:
from sm_model_classes import Semantic_Layer, SemanticModel, Entity , Dimension, Measure
import json 
from snowflake.snowpark import Session, DataFrame


connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()

SL_JF= self=Semantic_Layer(session, 'analytics','SL_TEST') #Instance de la classe semantic layer

SL_JF.create_semantic_layer() #Créer le Sematic_layer: Schéma dans snowflake 

 create schema if not exists analytics.SL_TEST 
 [Row(status='SL_TEST already exists, statement succeeded.')] 

 create tag if not exists analytics.SL_TEST.SNOW_MS_SL_TEST_OBJECT 
 [Row(status='SNOW_MS_SL_TEST_OBJECT already exists, statement succeeded.')] 

 create tag if not exists analytics.SL_TEST.SNOW_MS_SL_TEST_SEMANTIC_MODEL_METADATA 
 [Row(status='SNOW_MS_SL_TEST_SEMANTIC_MODEL_METADATA already exists, statement succeeded.')] 

 create tag if not exists analytics.SL_TEST.SNOW_MS_SL_TEST_SEMANTIC_MODEL_DIMENSIONS 
 [Row(status='SNOW_MS_SL_TEST_SEMANTIC_MODEL_DIMENSIONS already exists, statement succeeded.')] 

 create tag if not exists analytics.SL_TEST.SNOW_MS_SL_TEST_MEASURES 
 [Row(status='SNOW_MS_SL_TEST_MEASURES already exists, statement succeeded.')] 



2) Définition des Entities

In [3]:
e = Entity(entity_name='ORDERS', join_keys='order_id',desc='Primary Key for orders')   #entité "Orders" définie par la clé order_id
SL_JF.register_entity(e)    #enregistrer l'entité orders dans le semantic layer SL_JF


CREATE TAG IF NOT EXISTS analytics.SL_TEST.SNOW_METRIC_STORE_ENTITY_ORDERS
                    ALLOWED_VALUES 'order_id'
                    COMMENT = 'Primary Key for orders' 
 [Row(status='SNOW_METRIC_STORE_ENTITY_ORDERS already exists, statement succeeded.')] 



In [3]:

SL_JF.get_entities()   #lister les entités du semantic layer
#penser à ajouterle user & date de mise à jour 

---------------------------------------------------------
|"ENTITY_NAME"  |"KEYS"        |"DESCRIPTION"           |
---------------------------------------------------------
|ORDERS         |["order_id"]  |Primary Key for orders  |
---------------------------------------------------------



Ajouter des modeles au SL: 

In [4]:
from sm_model_classes import Semantic_Layer, SemanticModel, Entity , Dimension, Measure


SM=SemanticModel('orders','RAW','DMT_SCHEMA','FCT_ORDERS',e,'order_date','Order fact table. This table’s grain is one row per order')


SL_JF.register_semantic_model(SM)

CREATE or replace view analytics.SL_TEST.orders
                COMMENT = 'Order fact table. This table’s grain is one row per order'
                TAG (
                    analytics.SL_TEST.SNOW_MS_SL_TEST_OBJECT = '{"type": "sm_view"}',
analytics.SL_TEST.SNOW_MS_SL_TEST_SEMANTIC_MODEL_METADATA = '{"entities": ["ORDERS"], "timestamp_col": "order_date"}',
analytics.SL_TEST.SNOW_METRIC_STORE_ENTITY_ORDERS = 'order_id'
                )
                AS (select * from RAW.DMT_SCHEMA.FCT_ORDERS) 
 [Row(status='View ORDERS successfully created.')] 



Ajouter les dimensions du Modéle sémantique SM

In [5]:
dimension=Dimension('CUSTOMER_ID','categorical','','the customer id')    #instance
SL_JF.register_dimension(SM,dimension)   #enregistrer la dimension du modèle SM



ALTER TABLE analytics.SL_TEST.orders MODIFY COLUMN CUSTOMER_ID SET TAG analytics.SL_TEST.SNOW_MS_SL_TEST_SEMANTIC_MODEL_DIMENSIONS = '{"type":"categorical", "type_params":,"descripton":the customer id}';
 
 [Row(status='Statement executed successfully.')] 



In [6]:
SL_JF.get_dimensions(SM)

use schema analytics.SL_TEST;  
 [Row(status='Statement executed successfully.')] 

---------------------------------------------------------------------------------------
|"SEMANTIC_MODEL"  |"DIMENSION"  |"DIMENSION_DETAILS"                                 |
---------------------------------------------------------------------------------------
|ORDERS            |CUSTOMER_ID  |{"type":"categorical", "type_params":,"descript...  |
---------------------------------------------------------------------------------------



In [7]:


measure=Measure('order_total','The total amount for each order including taxes.','sum','amount')#créer une instance mesure 

   
SL_JF.register_measure(SM,measure)     #affecter la mesure measure au SM (colonne utilisée : amount)




ALTER TAG analytics.SL_TEST.SNOW_MS_SL_TEST_MEASURES ADD ALLOWED_VALUES '{orders:order_total}';
  
 [Row(status='Statement executed successfully.')] 

create tag if not exists analytics.SL_TEST.SNOW_MS_MEASURE_orders_order_total COMMENT='order_total tag';
  
 [Row(status='Tag SNOW_MS_MEASURE_ORDERS_ORDER_TOTAL successfully created.')] 

 ALTER TABLE analytics.SL_TEST.orders SET TAG analytics.SL_TEST.SNOW_MS_MEASURE_orders_order_total = '{"description":"The total amount for each order including taxes.", "aggregation":"sum", "expression":"amount"}'; 
 [Row(status='Statement executed successfully.')] 

use schema analytics.SL_TEST;  
 [Row(status='Statement executed successfully.')] 

---------------------------------------------------------------------------------------------------------------------------------------------
|"SEMANTIC_MODEL"  |"MEASURE"                           |"DESCRIPTION"                                       |"AGGREGATION"  |"EXPRESSION"  |
-------------------------

In [8]:
SL_JF.get_measures(SM)

use schema analytics.SL_TEST;  
 [Row(status='Statement executed successfully.')] 

---------------------------------------------------------------------------------------------------------------------------------------------
|"SEMANTIC_MODEL"  |"MEASURE"                           |"DESCRIPTION"                                       |"AGGREGATION"  |"EXPRESSION"  |
---------------------------------------------------------------------------------------------------------------------------------------------
|ORDERS            |SNOW_MS_MEASURE_ORDERS_ORDER_TOTAL  |"The total amount for each order including taxes."  |"sum"          |"amount"      |
---------------------------------------------------------------------------------------------------------------------------------------------

